# 5. Implement Hybrid Swarm Intelligence for Feature Selection (Filled Implementation)
Concrete hybrid ACO+PSO+ABC+MWPA integration for binary feature selection.

In [ ]:

import numpy as np
import time

def _binarize_position(position, threshold=0.5):
    sigmoid_output = 1 / (1 + np.exp(-10 * (position - 0.5)))
    return (sigmoid_output > threshold).astype(int)

class HybridSwarmFeatureSelector:
    def __init__(self, n_features, n_agents=30, max_iter=50, alpha_fitness=0.05,
                 w=0.9, c1=1.5, c2=1.5, evap_rate=0.1, deposit_amount=0.1, limit=5, beta=1.5):
        self.n_features = n_features
        self.n_agents = n_agents
        self.max_iter = max_iter
        self.alpha_fitness = alpha_fitness
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.evap_rate = evap_rate
        self.deposit_amount = deposit_amount
        self.limit = limit
        self.beta = beta

        # Initialize
        self.agents = np.random.rand(n_agents, n_features)
        self.velocities = np.zeros_like(self.agents)
        self.pbest_pos = self.agents.copy()
        self.pbest_fit = np.full(n_agents, -np.inf)
        self.gbest_pos = np.zeros(n_features)
        self.gbest_fit = -np.inf
        self.pheromones = np.ones(n_features) * 0.1
        self.trials = np.zeros(n_agents, dtype=int)
        self.fitness_history = []
        self.computational_time = 0

    def initialize(self, X_train, y_train, X_val, y_val):
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        fits = []
        for i in range(self.n_agents):
            mask = _binarize_position(self.agents[i])
            f = feature_selection_fitness(mask, X_train, y_train, X_val, y_val, alpha=self.alpha_fitness)
            self.pbest_fit[i] = f
            fits.append(f)
            if f > self.gbest_fit:
                self.gbest_fit = f
                self.gbest_pos = self.agents[i].copy()
        self.fitness_history.append(self.gbest_fit)

    def update(self):
        for i in range(self.n_agents):
            r1, r2 = np.random.rand(self.n_features), np.random.rand(self.n_features)
            # PSO term
            pso_vel = self.w * self.velocities[i]                       + self.c1 * r1 * (self.pbest_pos[i] - self.agents[i])                       + self.c2 * r2 * (self.gbest_pos - self.agents[i])
            # MWPA term
            A = 2 * (1 - len(self.fitness_history)/self.max_iter) * np.random.rand()
            D = np.abs(2 * np.random.rand() * self.gbest_pos - self.agents[i])
            mwpa = A * (D ** self.beta)
            # ACO influence (smooth pheromone push)
            aco = (self.pheromones / self.pheromones.sum()) - 0.5
            # Combine
            vel = pso_vel + mwpa + 0.1 * aco
            pos = self.agents[i] + vel
            pos = np.clip(pos, 0, 1)
            mask = _binarize_position(pos)
            f = feature_selection_fitness(mask, self.X_train, self.y_train,
                                          self.X_val, self.y_val, alpha=self.alpha_fitness)
            # Update trial or scout
            if f > self.pbest_fit[i]:
                self.pbest_fit[i] = f
                self.pbest_pos[i] = pos
                self.trials[i] = 0
            else:
                self.trials[i] += 1
                if self.trials[i] > self.limit:
                    pos = np.random.rand(self.n_features)
                    self.trials[i] = 0
            # Commit
            self.agents[i] = pos
            self.velocities[i] = vel

            if f > self.gbest_fit:
                self.gbest_fit = f
                self.gbest_pos = pos
        # Update pheromones
        self.pheromones *= (1 - self.evap_rate)
        best_mask = _binarize_position(self.gbest_pos)
        self.pheromones[best_mask == 1] += self.deposit_amount

    def run(self, X_train, y_train, X_val, y_val):
        start = time.time()
        self.initialize(X_train, y_train, X_val, y_val)
        for iter in range(self.max_iter):
            self.update()
            self.fitness_history.append(self.gbest_fit)
        self.computational_time = time.time() - start
        return _binarize_position(self.gbest_pos), self.fitness_history, self.computational_time


# 9. Benchmark Function Analysis (Filled Implementation)
Concrete hybrid optimizer for continuous benchmark functions.

In [ ]:

import numpy as np
import time

class HybridSwarmOptimizer:
    def __init__(self, obj_func, bounds, n_agents=30, max_iter=50,
                 w=0.9, c1=1.5, c2=1.5, evap_rate=0.1, deposit_amount=0.1, limit=5, beta=1.5):
        self.f = obj_func
        self.bounds = np.array(bounds)
        self.n_agents = n_agents
        self.max_iter = max_iter
        self.w, self.c1, self.c2 = w, c1, c2
        self.evap_rate, self.deposit_amount = evap_rate, deposit_amount
        self.limit, self.beta = limit, beta
        d = len(bounds)
        self.agents = np.random.uniform(self.bounds[:,0], self.bounds[:,1], (n_agents, d))
        self.velocities = np.zeros_like(self.agents)
        self.pbest_pos = self.agents.copy()
        self.pbest_val = np.full(n_agents, np.inf)
        self.gbest_pos = None
        self.gbest_val = np.inf
        self.pheromones = np.ones(d) * 0.1
        self.trials = np.zeros(n_agents, int)
        self.history = []
        self.computational_time = 0

    def initialize(self):
        vals = [self.f(x) for x in self.agents]
        for i, v in enumerate(vals):
            self.pbest_val[i] = v
            if v < self.gbest_val:
                self.gbest_val = v
                self.gbest_pos = self.agents[i].copy()
        self.history.append(self.gbest_val)

    def update(self):
        for i in range(self.n_agents):
            r1, r2 = np.random.rand(), np.random.rand()
            # PSO
            vel = self.w*self.velocities[i]                   + self.c1*r1*(self.pbest_pos[i]-self.agents[i])                   + self.c2*r2*(self.gbest_pos-self.agents[i])
            # MWPA
            A = 2*(1 - len(self.history)/self.max_iter)*np.random.rand()
            D = np.abs(2*np.random.rand()*self.gbest_pos - self.agents[i])
            mwpa = -A*(D**self.beta)
            # ACO (Continuous): small drift along pheromone gradient
            aco = (self.pheromones - self.pheromones.mean())*0.01
            vel = vel + mwpa + aco
            pos = self.agents[i] + vel
            # Clip
            pos = np.clip(pos, self.bounds[:,0], self.bounds[:,1])
            val = self.f(pos)
            # Scout
            if val < self.pbest_val[i]:
                self.pbest_val[i] = val
                self.pbest_pos[i] = pos.copy()
                self.trials[i] = 0
            else:
                self.trials[i] += 1
                if self.trials[i] > self.limit:
                    pos = np.random.uniform(self.bounds[:,0], self.bounds[:,1])
                    self.trials[i] = 0
            self.agents[i], self.velocities[i] = pos, vel
            if val < self.gbest_val:
                self.gbest_val = val
                self.gbest_pos = pos.copy()
        # update pheromones
        self.pheromones *= (1-self.evap_rate)
        best_idx = np.argmin(self.pbest_val)
        self.pheromones += self.deposit_amount * (self.pbest_pos[best_idx] - self.bounds[:,0])/(self.bounds[:,1]-self.bounds[:,0])

    def run(self):
        start = time.time()
        self.initialize()
        for _ in range(self.max_iter):
            self.update()
            self.history.append(self.gbest_val)
        self.computational_time = time.time() - start
        return self.gbest_pos, self.gbest_val, self.history, self.computational_time
